In [122]:
import os
import pickle
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.utils import resample
from sklearn.linear_model import Lasso, Ridge, ElasticNet, LinearRegression
from sklearn.preprocessing import StandardScaler, MinMaxScaler, PolynomialFeatures, RobustScaler, OneHotEncoder
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_absolute_error, r2_score, mean_squared_error
from sklearn.pipeline import make_pipeline
from sklearn.compose import ColumnTransformer, make_column_transformer

In [123]:
os.chdir('/home/utilisateur/projet/Assurance/Assurance_FE/')
print(os.listdir())
dataset_path = "Clean_Dataset_Brief.csv"
df_clean = pd.read_csv(dataset_path)

['NoteBook_Modelisation.ipynb', 'streamlit.py', 'OIG.jpeg', 'Clean_Dataset_Brief.csv', '.venv', 'BroullonEli_Cleaned.ipynb', 'NoteBook_Netoyage.ipynb', 'modele.pkl', '.git', 'Guide.txt', 'note.txt', 'Dataset_Brief.csv', 'README.md', 'BroullonEli.ipynb', 'BrouillonFlo.ipynb', 'requirements.txt', 'NoteBook_Analyse.ipynb', '.gitignore']


In [124]:
# df_clean.describe()

Selection des features pour prédiction.

Définir target y.

In [125]:
# selected_features = ['age', 'Jeune', 'Adulte', 'Adulte_moyen', 'Senior', 'Très_senior', 'sex', 'is_female', 'is_male', 'bmi', 'Insuffisance pondérale', 'Poids_normal', 'Surpoids', 'Obésité_de_classe_I_(modérée)', 'Obésité_de_classe_II_(sévère)', 'children', 'children_0', 'children_1', 'children_2', 'children_3', 'children_4', 'children_5', 'smoker','is_smoker', 'is_not_smoker', 'is_southwest', 'is_southeast', 'is_northwest', 'is_northeast']

# x = df_clean[selected_features]
# y = df_clean['charges']

Faire le split train/test avec pour ratio commun is_smoker.

In [126]:
# X_train, X_test, y_train, y_test = train_test_split(x, y, shuffle=True, train_size=0.85, random_state=42, stratify=x['smoker'])

In [127]:
# param_grid = {'lasso__alpha': list(range(230, 244))}

# model = make_pipeline(
#     StandardScaler(),
#     PolynomialFeatures(degree=2),
#     Lasso(random_state=42, max_iter=100000)
# )

# grid_search = GridSearchCV(model, param_grid, cv=5)
# grid_search.fit(X_train, y_train)
# best_alpha = grid_search.best_params_['lasso__alpha']
# y_pred = grid_search.predict(X_test)

# r2 = r2_score(y_test, y_pred)
# mse = mean_squared_error(y_test, y_pred)
# rmse = np.sqrt(mse)

# print(grid_search.best_params_)
# print("Performance du modèle final:")
# print(f"R2 Score: {r2:.4f}")
# print(f"RMSE: {rmse:.2f}")


In [128]:
# model.fit(X_train, y_train)
# residuals_train = y_train - model.predict(X_train)
# outlier_indices_train = np.where(np.abs(residuals_train) > 3000)[0]

# print("Indices des valeurs aberrantes dans l'ensemble d'entraînement :", outlier_indices_train)
# print("Nombre d'indices trouvés :", len(outlier_indices_train))

In [129]:
# Filtrer les valeurs aberrantes de X_train et y_train_cleaned
# X_train_cleaned = X_train.iloc[~X_train.index.isin(outlier_indices_train)]
# y_train_cleaned = y_train.iloc[~y_train.index.isin(outlier_indices_train)]

In [130]:
data_path = "Dataset_Brief.csv"
df = pd.read_csv(data_path)

In [131]:
missing_rate = df.isna().sum()/df.shape[0]
missing_rate
df = df.dropna()
duplicates = df.duplicated()
duplicates_data = df[duplicates]
df = df.drop_duplicates(subset=["age", "sex", "bmi", "children", "smoker", "region"], keep='first')

In [132]:
selected_features = ['age','sex' ,'bmi', 'children', 'smoker', 'region']
X = df[selected_features]
y = df['charges']

In [133]:
X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle=True, train_size=0.85, random_state=42, stratify=X['smoker'])

In [134]:
num_col = list(X.select_dtypes(include=[float,int]).columns)
cat_col = list(X.select_dtypes(include=[object]).columns)

onehotscale_pipeline = make_pipeline(OneHotEncoder(), RobustScaler(with_centering=False))
scale_pipeline = make_pipeline(RobustScaler(with_centering=False))

preprocessing = ColumnTransformer(
    transformers=[
        ('categorical', onehotscale_pipeline, cat_col),
        ('numerical', scale_pipeline, num_col)]
)

param_grid = {'lasso__alpha': list(range(10, 55))}


polyscale_pipeline = make_pipeline(PolynomialFeatures(2))

my_final_pipeline = make_pipeline(preprocessing, polyscale_pipeline)
my_final_pipeline.fit(X_train)

model = make_pipeline(
    my_final_pipeline,
    Lasso(random_state=42, max_iter=100000)
)

grid_search = GridSearchCV(model, param_grid,cv =5)
grid_search.fit(X_train, y_train)
grid_score = grid_search.score(X_train, y_train)

best_alpha = grid_search.best_params_['lasso__alpha']

y_pred = grid_search.predict(X_test)

r2_cleaned = r2_score(y_test, y_pred)
mse_cleaned = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
rmse_cleaned = np.sqrt(mse_cleaned)

print("Performance du modèle après suppression des valeurs extrêmes:")
print(grid_search.best_params_)
print(f"R2 Score: {r2_cleaned:.4f}")
print(f"MAE: {mae:.2f}")
print(f"RMSE: {rmse_cleaned:.2f}")
 
with open('modele.pkl', 'wb') as file:
    pickle.dump(grid_search, file)

Performance du modèle après suppression des valeurs extrêmes:
{'lasso__alpha': 15}
R2 Score: 0.8614
MAE: 2725.26
RMSE: 4489.57
